Computing DWI files from the bias corrected inputs using the functions defined in DWI_functions.py

In [1]:
import dask
from dask.distributed import Client, wait
from dask import delayed

client = Client(n_workers=7, threads_per_worker=1) 
#client = Client()

client

/g/data/xp65/public/apps/med_conda/envs/analysis3-24.12/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35999 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/35999/status,
Dashboard: /proxy/35999/status,Workers: 7
Total threads: 7,Total memory: 251.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40597,Workers: 7
Dashboard: /proxy/35999/status,Total threads: 7
Started: Just now,Total memory: 251.19 GiB
Comm: tcp://127.0.0.1:39127,Total threads: 1
Dashboard: /proxy/42207/status,Memory: 35.88 GiB
Nanny: tcp://127.0.0.1:38113,


2025-09-26 12:22:19,909 - distributed.semaphore - WARNING - Tried to release Lock or Semaphore but it was already released: name='/g/data/ia39/ncra/bushfire/dwi/EC-Earth3/ssp370/r1i1p1f1/BARPA-R/v1-r1/day/ssp370_EC-Earth3_BARPA-R_QME_gwl1.2_faux_ffdi.nc', lease_id='af87c0ac6278419b96fb3e77d1947672'. This can happen if the Lock or Semaphore timed out before.
2025-09-26 14:05:08,461 - distributed.semaphore - WARNING - Tried to release Lock or Semaphore but it was already released: name='/g/data/ia39/ncra/bushfire/dwi/EC-Earth3/ssp370/r1i1p1f1/BARPA-R/v1-r1/day/ssp370_EC-Earth3_BARPA-R_QME_gwl1.5_dwi.nc', lease_id='a5662cc537e74d5fbe50bcc386c380f0'. This can happen if the Lock or Semaphore timed out before.
2025-09-26 14:06:50,030 - distributed.semaphore - WARNING - Tried to release Lock or Semaphore but it was already released: name='/g/data/ia39/ncra/bushfire/dwi/EC-Earth3/ssp370/r1i1p1f1/BARPA-R/v1-r1/day/ssp370_EC-Earth3_BARPA-R_QME_gwl1.5_dwi.nc', lease_id='79e73d920e204668a171bd00fb

In [2]:
#import all the stuff
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import glob
import sys
sys.path.append("/g/data/mn51/users/nb6195/project/dwi/DWI")
import DWI_functions as DWI
sys.path.append("/g/data/mn51/users/nb6195/project/gwls/")
import gwl

Imported functions are:
- calc_dpd(ds_temp, ds_rh)
- calc_dwi(ds_temp, ds_rh, ds_wind_sp, A, B, C, D)
- calc_p_ffdi(ds_dwi)
- calc_faux_ffdi(ds_dwi, ds_temp)

In [3]:
#Set parameters
CMIP='CMIP6'
#AGENCY = 'CSIRO' 
#RCM = 'CCAM-v2203-SN'
AGENCY = 'BOM' 
RCM = 'BARPA-R'

#GCM = 'ACCESS-CM2' ensemble = 'r4i1p1f1' #Done
#GCM = 'ACCESS-ESM1-5' ensemble = 'r6i1p1f1' #Done
GCM = 'EC-Earth3' 
ensemble = 'r1i1p1f1' #Done
#GCM = 'MPI-ESM1-2-HR' ensemble = 'r1i1p1f1' #BOM done, no CSIRO
#GCM = 'CESM2' ensemble = 'r11i1p1f1' #Done
#GCM = 'CMCC-ESM2' ensemble = 'r1i1p1f1' #Done
#GCM = 'NorESM2-MM' ensemble = 'r1i1p1f1' #Done
#GCM = 'CNRM-ESM2-1' ensemble = 'r1i1p1f2' #CSIRO Done, no BOM

#pathway = 'ssp126'
pathway = 'ssp370'

#bc_method = 'MRNBC'
bc_method = 'QME'

ddir = f"/g/data/kj66/CORDEX/output-{CMIP}/bias-adjusted-output/AUST-05i/{AGENCY}/{GCM}"
output_dir = '/g/data/ia39/ncra/bushfire/dwi/'

In [4]:
var1 = 'tasmaxAdjust'
var2 = 'hursminAdjust'
var3 = 'sfcWindmaxAdjust'

In [5]:
#read in files

#tasmax
infiles1a=glob.glob(ddir+f'/historical/{ensemble}/{RCM}/v1-r1-ACS-{bc_method}-BARRAR2-1980-2022/day/{var1}/v20241216/{var1}_AUST-05i_{GCM}_historical_{ensemble}_{AGENCY}_{RCM}_v1-r1-ACS-{bc_method}-BARRAR2-1980-2022_day_*.nc')
infiles1b=glob.glob(ddir+f'/{pathway}/{ensemble}/{RCM}/v1-r1-ACS-{bc_method}-BARRAR2-1980-2022/day/{var1}/v20241216/{var1}_AUST-05i_{GCM}_{pathway}_{ensemble}_{AGENCY}_{RCM}_v1-r1-ACS-{bc_method}-BARRAR2-1980-2022_day_*.nc')
tasmax_master_ds = xr.open_mfdataset(infiles1a + infiles1b)

#hursmin
infiles2a=glob.glob(ddir+f'/historical/{ensemble}/{RCM}/v1-r1-ACS-{bc_method}-BARRAR2-1980-2022/day/{var2}/v20241216/{var2}_AUST-05i_{GCM}_historical_{ensemble}_{AGENCY}_{RCM}_v1-r1-ACS-{bc_method}-BARRAR2-1980-2022_day_*.nc')
infiles2b=glob.glob(ddir+f'/{pathway}/{ensemble}/{RCM}/v1-r1-ACS-{bc_method}-BARRAR2-1980-2022/day/{var2}/v20241216/{var2}_AUST-05i_{GCM}_{pathway}_{ensemble}_{AGENCY}_{RCM}_v1-r1-ACS-{bc_method}-BARRAR2-1980-2022_day_*.nc')
hursmin_master_ds = xr.open_mfdataset(infiles2a + infiles2b)

#wind
infiles3a=glob.glob(ddir+f'/historical/{ensemble}/{RCM}/v1-r1-ACS-{bc_method}-BARRAR2-1980-2022/day/{var3}/v20241216/{var3}_AUST-05i_{GCM}_historical_{ensemble}_{AGENCY}_{RCM}_v1-r1-ACS-{bc_method}-BARRAR2-1980-2022_day_*.nc')
infiles3b=glob.glob(ddir+f'/{pathway}/{ensemble}/{RCM}/v1-r1-ACS-{bc_method}-BARRAR2-1980-2022/day/{var3}/v20241216/{var3}_AUST-05i_{GCM}_{pathway}_{ensemble}_{AGENCY}_{RCM}_v1-r1-ACS-{bc_method}-BARRAR2-1980-2022_day_*.nc')
wind_sp_master_ds = xr.open_mfdataset(infiles3a + infiles3b)

In [6]:
#Vesta DWI: fire spread rate, expect up to 10% of wind speed in 
#extreme conditions (very dry M. Cruz results).
#Values here from the paper provide an excellent approximation for 
#windspeeds between 10 and 80 km\ h^{-1}, using the longest unburned 
#fuel settings (10+ years) provided in Table 6 of Cheney et al. (2012).  
A = 5
B = 20
C = 0
D = 25

In [30]:
#Extract time period corresponding to the chosen GWL for tasmax and rh
chosen_gwl = '2.0'

gwl_tasmax = gwl.get_GWL_timeslice(tasmax_master_ds,CMIP,GCM,ensemble,pathway,GWL=chosen_gwl)[var1]
gwl_rh = gwl.get_GWL_timeslice(hursmin_master_ds,CMIP,GCM,ensemble,pathway,GWL=chosen_gwl)[var2]
gwl_wind_sp = gwl.get_GWL_timeslice(wind_sp_master_ds,CMIP,GCM,ensemble,pathway,GWL=chosen_gwl)[var3]

In [31]:
#function testing
gwl_dwi = DWI.calc_dwi(gwl_tasmax, gwl_rh, gwl_wind_sp, chosen_gwl, A, B, C, D)['dwi']

In [33]:
file_name_dwi = pathway + '_' + GCM + '_' + RCM + '_' + bc_method + '_gwl' + chosen_gwl + '_dwi.nc'
output_file_location = output_dir + GCM + '/' + pathway + '/' + ensemble + '/' + RCM + '/v1-r1/day/' + file_name_dwi
gwl_dwi.to_netcdf(output_file_location, engine='netcdf4')
print(output_file_location)

/g/data/ia39/ncra/bushfire/dwi/EC-Earth3/ssp370/r1i1p1f1/BARPA-R/v1-r1/day/ssp370_EC-Earth3_BARPA-R_QME_gwl2.0_dwi.nc


In [34]:
gwl_p_ffdi = DWI.calc_p_ffdi(gwl_dwi, chosen_gwl)

In [35]:
file_name_p_ffdi = pathway + '_' + GCM + '_' + RCM + '_' + bc_method + '_gwl' + chosen_gwl + '_p_ffdi.nc'
output_file_location = output_dir + GCM + '/' + pathway + '/' + ensemble + '/' + RCM + '/v1-r1/day/' + file_name_p_ffdi
gwl_p_ffdi.to_netcdf(output_file_location, engine='netcdf4')
print(output_file_location)

/g/data/ia39/ncra/bushfire/dwi/EC-Earth3/ssp370/r1i1p1f1/BARPA-R/v1-r1/day/ssp370_EC-Earth3_BARPA-R_QME_gwl2.0_p_ffdi.nc


In [36]:
gwl_faux_ffdi = DWI.calc_faux_ffdi(gwl_dwi, gwl_tasmax, chosen_gwl)

In [40]:
file_name_faux_ffdi = pathway + '_' + GCM + '_' + RCM + '_' + bc_method + '_gwl' + chosen_gwl + '_faux_ffdi.nc'
output_file_location = output_dir + GCM + '/' + pathway + '/' + ensemble + '/' + RCM + '/v1-r1/day/' + file_name_faux_ffdi
gwl_faux_ffdi.to_netcdf(output_file_location, engine='netcdf4')
print(output_file_location)

/g/data/ia39/ncra/bushfire/dwi/EC-Earth3/ssp370/r1i1p1f1/BARPA-R/v1-r1/day/ssp370_EC-Earth3_BARPA-R_QME_gwl2.0_faux_ffdi.nc
